# A GENTLE INTRODUCTION TO ```torch.autograd```
- ```torch.autograd``` 는 인공신경망 학습을 위한 파이토치의 자동 미분 엔진임
- 이 섹션에서는 **autograd 가 어떻게 인공신경망의 학습을 돕는지에 대한 컨셉을 이해해볼 것**

---
# 1. Background
- 인공신경망은 입력 함수에 대해 적용되는 밀집 함수의 집합임
- 이 함수는 weight 와 bias 로 이루어진 **파라미터로 정의됨**
- 이 파라미터는 파이토치에서 텐서의 형태로 저장됨

>**인공신경망의 학습은 두 단계로 나뉨**
>1. **Forward Propagation** : 인공신경망은 정답값에 대한 최선의 출력을 만듦. 입력을 받으면 여러 함수를 거쳐 최종 출력을 반환함
>2. **Backward Propagation** : 위 단계에서 발생한 에러에 비례하여 파라미터를 조정함. 출력으로부터 입력단계까지 역전파해가며 각 파라미터에 대한 derivatives 를 구함. 구한 derivatives 를 기반으로한 gradient descent 를 이용하여 파라미터를 조정.

---
# 2. Usage in PyTorch
- 간단한 1 strp 학습을 예로 들겠음
- 먼저 ```torchvision``` 을 통해 사전학습된 resnet18 모델을 로드
- 해당 모델의 입력을 임의로 생성 (채널 3, 가로 세로 64) / (라벨 임의의 정수)

In [1]:
import torch, torchvision

model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to C:\Users\gus8c/.cache\torch\hub\checkpoints\resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

- 이제 임의로 생성한 입력을 모델에 넣어 출력을 만들어 보겠음
- 이 작업은 **forward pass** 로 진행됨

In [4]:
pred = model(data)
print(pred.size())

torch.Size([1, 1000])


- 모델의 예측과 정답값을 이용하여 에러(```loss```)를 구함
- 에러를 구하면 네트워크에 **역전파** 함. 이는 에러 텐서의 ```.backward()``` 메서드를 호출하면 됨
- **Autograd** 는 모델의 각 파라미터의 ```.grad``` 특성에 gradient 를 구하여 저장함

In [6]:
loss = (pred - labels).sum()
loss.backward()

- 이제 **optimizer** 를 정의하여 학습할 파라미터를 로드함
- 여기선 **SGD** 를 사용. **학습률**은 0.01, **모멘텀**은 0.9 로 설정

In [7]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

- 마지막으로 ```.step()``` 메서드를 호출하여 gradient descent 를 실행함
- 호출하면 optimizer 는 각 파라미터의 ```.grad``` 특성에 저장된 gradient 를 이용하여 업데이트함

In [8]:
optim.step()

---
# 3. Difference in Autograd
- ```autograd``` 가 어떻게 gradient 를 수집하는지 보겠음
- 예시로 ```requires_grea=True``` 로 설정한 두 텐서를 만듦. 이제 이 두 텐서가 사용되는 모든 연산은 ```autograd``` 가 추적할 것임

In [9]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

- 위에서 만든 ```a, b``` 로 이루어진 새로운 텐서 ```Q``` 를 만듦

In [10]:
Q = 3*a**3 - b**2
print(Q)

tensor([-12.,  65.], grad_fn=<SubBackward0>)


>```a, b``` 를 인공신경망의 파라미터, ```Q``` 를 그 신경망의 에러라고 가정
>- ```Q``` 에 대해 ```.backward()```를 호출하면 사용된 파라미터인 ```a, b``` 의 gradient 가 구해짐
>- 이 gradient 는 각 파라미터의 ```.grad()``` 특성에 저장됨
>- **이때 에러가 1차원이 아닌 벡터이므로 ```Q.backward()``` 메서드에 ```gradient``` 매개변수를 전달해야 함**. ```gradient``` 매개변수엔 ```Q``` 와 같은 shape 를 가진 텐서를 넣으면 됌. **이는 Q 자기 자신에 대한 gradient 와 같음**
>- ```Q.sum().backward()``` 형식으로 벡터를 스칼라로 바꿔서 진행할 수도 있음

In [12]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

- 역전파로 각 파라미터에 대한 gradient 가 계산됨
- 각 파라미터의 ```.grad``` 특성을 확인

In [14]:
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


---
# 4. Exclusion from the DAG
- ```torch.autograd``` 는 ```requires_grad=True``` 로 설정된 모든 텐서들이 관여한 연산을 추적함
- 추적이 필요하지 않은 텐서는 ```requires_grad=False``` 로 설정하면 됨. 그러면 gradient computation DAG 로부터 배제됨

In [18]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(a.requires_grad)

b = x + z
print(b.requires_grad)

False
True


## 4.1 Frozen Parameters
- 인공신경망에서 gradient 를 계산하지 않는 파라미터를 **frozen parameters** 라고 함
- 특히 사전학습된 모델을 사용할 때 모델의 일부분을 **freeze** 하는 것은 연산을 줄일 수 있어 효율적임


>또는 **사전학습된 모델을 파인튜닝** 할 때 매우 유용함
>- 최종 출력층을 제외한 모든 층을 freeze 하고
>- 마지막 출력층만 따로 학습하는 식으로 진행

In [23]:
from torch import nn, optim

model = torchvision.models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

- 새롭게 분류할 타겟이 **10개의 클래스**를 갖고 있다면 출력이 10개가 되야함
- 여기서 사용한 모델은 출력층이 1000 개임
- 여기선 모델의 출력층인 ```model.fc``` 층을 수정하여 구현하겠음 (기본적으로 unfrozen)

In [25]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [26]:
model.fc = nn.Linear(512, 10)

- 이제 모델은 출력층을 제외하고 모든 파라미터가 frozen 상태임
- 에러를 구하고 gradient 를 구하면 출력층에 대해서만 나옴

In [27]:
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

>- 여기서 모든 파라미터를 optimizer 에 올렸지만 gradient 가 구해지는 파라미터는 unfrozen 파라미터 뿐이므로 안심